In [9]:
# Imports
import os
import json
import pandas as pd

In [11]:
# Directory containing JSON files
directory = './spotify_million_playlist_dataset/data'
# directory = './Test Playlist'
# Initialize data structures
playlists_data = []
unique_songs = {}  # Dictionary to store unique songs with URI as key
playlist_song_map = {}  # Mapping of playlist name to song URIs

# Read each JSON file and extract data
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            data = json.load(file)
            for playlist in data['playlists']:
                # Extract playlist-level features
                playlist_info = {
                    'name': playlist['name'],
                    'num_tracks': playlist['num_tracks'],
                    'num_albums': playlist['num_albums'],
                    'num_followers': playlist['num_followers'],
                    'num_edits': playlist['num_edits'],
                    'duration_ms': playlist['duration_ms'],
                    'num_artists': playlist['num_artists']
                }
                playlists_data.append(playlist_info)

                # Initialize song list for this playlist
                playlist_song_map[playlist['name']] = []

                # Extract and store unique track-level features
                for track in playlist['tracks']:
                    track_uri = track['track_uri']
                    if track_uri not in unique_songs:
                        unique_songs[track_uri] = {
                            'artist_name': track['artist_name'],
                            'track_name': track['track_name'],
                            'duration_ms': track['duration_ms']
                        }
                    playlist_song_map[playlist['name']].append(track_uri)

# Convert playlists data and unique songs to DataFrames
playlists_df = pd.DataFrame(playlists_data)
unique_songs_df = pd.DataFrame(unique_songs.values(), index=unique_songs.keys())

# Now, playlists_df, unique_songs_df, and playlist_song_map are ready for further processing